# Node Inputs

In [37]:
import json
from subprocess import call
import numpy as np

# Important parameters

# Total number of nodes
n = [4,8,16,32,64,100,256]

# Number of witnesses
w = [int(min(3*np.log2(x),x)) for x in n]
v = [int(min(4*np.log2(x),x)) for x in n]
u = [x//2 + 1 for x in w]

# Number of Samples
s = [int(min(4*np.log2(x),x)) for x in n]
th = [(2*x)//3 + 1 for x in s]

In [38]:
# Brachas's inputs

for i in range(len(n)):
    if n[i]%3 == 0:
        f = n[i]//3 - 1
    else:
        f = n[i]//3
    experiment = {"protocol": "bracha"}
    experiment["parameters"] = {"n": n[i], "f": f}
    
    # Serializing json
    json_object = json.dumps(experiment, indent=4)

    # Writing to sample.json
    with open("Experiments/brachaInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [39]:
# Reliable Accountability

# Fixed Parameters
wr = 10
vr = 10
recoveryTm = 60000000000
idSize = 256
numberBins = 16

for i in range(len(n)):
    experiment = {"protocol": "reliable_accountability"}
    experiment["parameters"] = {"n": n[i], "f": 0,
                               "w": w[i], "v": v[i], "wr": 10, "vr": 10,
                               "u": u[i], "recovery_timeout": recoveryTm,
                               "node_id_size": idSize, "number_of_bins": numberBins}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/witnessInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

In [40]:
# Scalable Broadcast

for i in range(len(n)):
    experiment = {"protocol": "scalable"}
    experiment["parameters"] = {"n": n[i], "f": 0,
                               "g_size": s[i], "e_size": s[i], "r_size": s[i], "d_size": s[i],
                               "e_threshold": th[i], "r_threshold": th[i], "d_threshold": th[i],
                                "clean_up_timeout": 10000000000}
    
    json_object = json.dumps(experiment, indent=4)

    with open("Experiments/scalableInput" + str(i) + ".json", "w") as outfile:
        outfile.write(json_object)

# Simulation config files

In [26]:
128//32

4

In [31]:
n = 128
step = (128//n)*4
step*8

32

In [33]:
dictFiles = {32:3, 64:4, 128:5}

In [34]:
dictFiles

{32: 3, 64: 4, 128: 5}

In [22]:
len([2*(i+1) for i in range(64)])

64

In [20]:
# Changes bellow are mainly on the system's throughput
# Transactions / second
#targetThr = [int((2**(1/2))**(2+i)) for i in range(15)]
targetThr = [2*(i+1) for i in range(64)]

# Requests are evengly split among nodes according to the formula  

#    thr = n * thr_n, where thr_n is a node's requests per second

totalTime = 180000000000 # 180 seconds in nanoseconds
baseDelay = 1000000000 # 1 second in nanoseconds
for i in range(len(n)):
    for th in targetThr:
        # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 10, "hostQueue": 10000,
                  "hostDelay": "5ms", "hostLoss": 1}
        
        config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}
        
        trDelay = (n[i]/th) * baseDelay
        totalTr = totalTime/trDelay
        
        config["simulationTime"] = int(totalTime/baseDelay) + 120
        config["numberTransactions"] = int(totalTr)
        config["transactionDelay"] = int(trDelay)
        
        json_object = json.dumps(config, indent=4)

        with open("Experiments/config" + str(i) + "_" + str(th) + ".json", "w") as outfile:
            outfile.write(json_object)

# Stress Test

In [35]:
for i in range(3):
    
        # Fixed Parameters
        config = {"topology": "singleTopo.json",
                  "hostBand": 2**(i+1), "hostQueue": 10000,
                  "hostDelay": "0ms", "hostLoss": 1}

        config["testNodes"] = {"inTestNodes": False, "outTestNodes": False,
                               "inTopoDelay": "25ms", "outTopoDelay": "25ms"}


        config["simulationTime"] = 250
        config["numberTransactions"] = -1
        config["transactionDelay"] = 180000000000

        json_object = json.dumps(config, indent=4)

        with open("Experiments/stress" + str(i) + ".json", "w") as outfile:
            outfile.write(json_object)